<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#imports-and-functions" data-toc-modified-id="imports-and-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>imports and functions</a></span></li><li><span><a href="#experiments" data-toc-modified-id="experiments-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>experiments</a></span><ul class="toc-item"><li><span><a href="#12" data-toc-modified-id="12-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>12</a></span></li><li><span><a href="#13" data-toc-modified-id="13-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>13</a></span></li><li><span><a href="#14" data-toc-modified-id="14-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>14</a></span></li><li><span><a href="#15" data-toc-modified-id="15-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>15</a></span></li><li><span><a href="#16" data-toc-modified-id="16-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>16</a></span><ul class="toc-item"><li><span><a href="#debug" data-toc-modified-id="debug-2.5.1"><span class="toc-item-num">2.5.1&nbsp;&nbsp;</span>debug</a></span></li></ul></li></ul></li></ul></div>

# imports and functions

In [ ]:
import collections
import pandas as pd
import os
import sys
import json
import matplotlib.pyplot as plt
from tqdm.auto import tqdm, trange

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torch.optim import lr_scheduler
# import torchmetrics

import datasets
from datasets import load_metric
from transformers import AutoConfig, AutoTokenizer, BertModel, RobertaModel
from transformers import BertForSequenceClassification
from transformers import TrainingArguments, Trainer

from sklearn.metrics import mean_squared_error, accuracy_score, precision_recall_fscore_support


In [ ]:
# https://github.com/huggingface/transformers/issues/5486
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
with open('../../data/xslue/tasks.json', 'r') as f:
    tasks = json.load(f)
tasks

{'CrowdFlower': 13,
 'DailyDialog': 7,
 'EmoBank_Valence': 1,
 'EmoBank_Arousal': 1,
 'EmoBank_Dominance': 1,
 'HateOffensive': 3,
 'PASTEL_age': 8,
 'PASTEL_country': 2,
 'PASTEL_education': 10,
 'PASTEL_ethnic': 10,
 'PASTEL_gender': 3,
 'PASTEL_politics': 3,
 'PASTEL_tod': 5,
 'SARC': 2,
 'SarcasmGhosh': 2,
 'SentiTreeBank': 1,
 'ShortHumor': 2,
 'ShortJokeKaggle': 2,
 'ShortRomance': 2,
 'StanfordPoliteness': 1,
 'TroFi': 2,
 'VUA': 2}

In [ ]:
class MyDataset(Dataset): 
    # currently it's a Mapping-style dataset. Not sure if a Iterable-style dataset will be better
    def __init__(self, tsv_file):
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.df = pd.read_csv(tsv_file, sep='\t')
        self.df = self.df.dropna()
        self.df = self.df.reset_index(drop=True)
        self.encodings = self.tokenizer(self.df['text'].tolist(), truncation=True, padding=True, max_length=128)
        if self.df['label'].dtype == 'float64':
            self.df['label'] = self.df['label'].astype('float32')
        self.labels = self.df['label'].tolist()
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item


In [ ]:
pearsonr = load_metric("pearsonr")
spearmanr = load_metric("spearmanr")

In [ ]:
# GPU memory usage: 6617 - 6680mb with bs 32
# bs 64 gives OOM
# bs 48 GPU memory 7894
batch_size = 32

In [ ]:
def train_baseline(task, freeze_bert=False):
    torch.cuda.empty_cache()
    model = None
    trainer = None 
    num_labels = tasks[task]
    
    data_folder = '../../data/xslue'
    train_dataset = MyDataset(f'{data_folder}/processed/train/{task}.tsv')
    test_dataset = MyDataset(f'{data_folder}/processed/test/{task}.tsv')
    valid_dataset = MyDataset(f'{data_folder}/processed/dev/{task}.tsv')
    
    singletaskbert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels) 
    if freeze_bert:
        for param in singletaskbert.bert.parameters():
            param.requires_grad = False
    result_folder = '../../result'
    training_args = TrainingArguments(
        output_dir=f"{result_folder}/baselines/{task+'_freezed' if freeze_bert else task}",   # output directory
        num_train_epochs=5,              # total number of training epochs
        per_device_train_batch_size=batch_size,  # batch size per device during training
        per_device_eval_batch_size=batch_size,   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir=f"{result_folder}/baselines/{task+'_freezed' if freeze_bert else task}/logs",  # directory for storing logs
#         logging_first_step = True, 
#         logging_steps=500,               # log & save weights each logging_steps
#         save_steps=500,
        evaluation_strategy="epoch",     # evaluate each `logging_steps`
        save_total_limit = 1,
        save_strategy = 'epoch',
        load_best_model_at_end=True, # decide on loss
    )
    
    if num_labels == 1:
        def compute_metrics(pred):
            predictions, labels = pred
            rmse = mean_squared_error(labels, predictions, squared=False)
            return {"rmse": rmse}
    elif num_labels == 2:
        def compute_metrics(pred):
            labels = pred.label_ids
            preds = pred.predictions.argmax(-1)
            precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
            acc = accuracy_score(labels, preds)
            return {
                'accuracy': acc,
                'f1': f1,
                'precision': precision,
                'recall': recall
            }
    else:
        def compute_metrics(pred):
            labels = pred.label_ids
            preds = pred.predictions.argmax(-1)
            precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
            acc = accuracy_score(labels, preds)
            return {
                'accuracy': acc,
                'f1': f1,
                'precision': precision,
                'recall': recall
            }
    
    trainer = Trainer(
        model=singletaskbert,   # the instantiated Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataset,         # training dataset
        eval_dataset=valid_dataset,          # evaluation dataset
#         test_dataset=test_dataset,            # test dataset
        compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    )
    trainer.train()

# experiments

## 12

In [ ]:
task = list(tasks.keys())[12]
train_baseline(task)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.296100,1.299696,0.470036,0.266865,0.404191,0.292907
2,1.264400,1.298557,0.471480,0.271082,0.441297,0.295143
3,1.101700,1.368624,0.429122,0.307349,0.328053,0.309603
4,0.728000,1.682379,0.427196,0.306962,0.323563,0.310037
5,0.448200,2.056610,0.419495,0.313106,0.323609,0.311995


***** Running Evaluation *****
  Num examples = 4155
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ../../result/baselines/PASTEL_tod/checkpoint-1039
Configuration saved in ../../result/baselines/PASTEL_tod/checkpoint-1039/config.json
Model weights saved in ../../result/baselines/PASTEL_tod/checkpoint-1039/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4155
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beha

In [ ]:
task = list(tasks.keys())[12]
train_baseline(task, freeze_bert=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_emb

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.426500,1.421628,0.401444,0.184231,0.165445,0.237363
2,1.422200,1.416608,0.407702,0.184889,0.173160,0.239852
3,1.414500,1.413576,0.409386,0.186414,0.173394,0.241145
4,1.421700,1.413384,0.410830,0.184860,0.177410,0.241053
5,1.415300,1.412030,0.408664,0.185410,0.174045,0.240421


***** Running Evaluation *****
  Num examples = 4155
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ../../result/baselines/PASTEL_tod_freezed/checkpoint-1039
Configuration saved in ../../result/baselines/PASTEL_tod_freezed/checkpoint-1039/config.json
Model weights saved in ../../result/baselines/PASTEL_tod_freezed/checkpoint-1039/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4155
  Batch size = 32
/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parame

## 13

In [ ]:
task = list(tasks.keys())[13]
train_baseline(task)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.560500,0.555968,0.714375,0.724823,0.698260,0.753487
2,0.487500,0.569209,0.720288,0.726626,0.709497,0.744604
3,0.353700,0.669095,0.712274,0.716726,0.704768,0.729097
4,0.205900,0.887727,0.702801,0.715671,0.685013,0.749201
5,0.124200,1.239568,0.702840,0.711281,0.690645,0.733188


***** Running Evaluation *****
  Num examples = 51410
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SARC/checkpoint-6427
Configuration saved in ../../result/baselines/SARC/checkpoint-6427/config.json
Model weights saved in ../../result/baselines/SARC/checkpoint-6427/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 51410
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SARC/checkpoint-12854
Configuration saved in ../../result/baselines/SARC/checkpoint-12854/config.json
Model weights saved in ../../result/baselines/SARC/checkpoint-12854/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 51410
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SARC/checkpoint-19281
Configuration saved in ../../result/baselines/SARC/checkpoint-19281/config.json
Model weights saved in ../../result/baselines/SARC/checkpoint-19281/pytorch_model.bin
Deleting older checkpoint [../../result/baselines/SARC/checkpoint-12854] 

In [ ]:
task = list(tasks.keys())[13]
train_baseline(task, freeze_bert=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.680100,0.672663,0.592531,0.603032,0.587035,0.619925
2,0.673100,0.666404,0.600350,0.599509,0.599860,0.599158
3,0.671500,0.663911,0.599300,0.610114,0.593228,0.627990
4,0.668300,0.662713,0.608247,0.576161,0.626442,0.533352
5,0.670400,0.662071,0.606691,0.596149,0.611589,0.581470


***** Running Evaluation *****
  Num examples = 51410
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SARC_freezed/checkpoint-6427
Configuration saved in ../../result/baselines/SARC_freezed/checkpoint-6427/config.json
Model weights saved in ../../result/baselines/SARC_freezed/checkpoint-6427/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 51410
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SARC_freezed/checkpoint-12854
Configuration saved in ../../result/baselines/SARC_freezed/checkpoint-12854/config.json
Model weights saved in ../../result/baselines/SARC_freezed/checkpoint-12854/pytorch_model.bin
Deleting older checkpoint [../../result/baselines/SARC_freezed/checkpoint-6427] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 51410
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SARC_freezed/checkpoint-19281
Configuration saved in ../../result/baselines/SARC_freezed/checkpoint-1928

## 14

In [ ]:
task = list(tasks.keys())[14]
train_baseline(task)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.013800,0.172218,0.977085,0.640000,0.761905,0.551724
2,0.010300,0.199137,0.977085,0.640000,0.761905,0.551724
3,0.005900,0.212433,0.977721,0.646465,0.780488,0.551724
4,0.002600,0.230788,0.977721,0.646465,0.780488,0.551724
5,0.001100,0.239562,0.977721,0.646465,0.780488,0.551724


***** Running Evaluation *****
  Num examples = 1571
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SarcasmGhosh/checkpoint-1244
Configuration saved in ../../result/baselines/SarcasmGhosh/checkpoint-1244/config.json
Model weights saved in ../../result/baselines/SarcasmGhosh/checkpoint-1244/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1571
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SarcasmGhosh/checkpoint-2488
Configuration saved in ../../result/baselines/SarcasmGhosh/checkpoint-2488/config.json
Model weights saved in ../../result/baselines/SarcasmGhosh/checkpoint-2488/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1571
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SarcasmGhosh/checkpoint-3732
Configuration saved in ../../result/baselines/SarcasmGhosh/checkpoint-3732/config.json
Model weights saved in ../../result/baselines/SarcasmGhosh/checkpoint-3732/pytorch_model.bin
Deleting o

In [ ]:
task = list(tasks.keys())[14]
train_baseline(task, freeze_bert=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.625000,0.550487,0.810312,0.207447,0.122642,0.672414
2,0.536100,0.504274,0.788670,0.198068,0.115169,0.706897
3,0.472600,0.473396,0.793762,0.201970,0.117816,0.706897
4,0.454600,0.452626,0.805856,0.211886,0.124620,0.706897
5,0.434900,0.450173,0.802673,0.209184,0.122754,0.706897


***** Running Evaluation *****
  Num examples = 1571
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SarcasmGhosh_freezed/checkpoint-1244
Configuration saved in ../../result/baselines/SarcasmGhosh_freezed/checkpoint-1244/config.json
Model weights saved in ../../result/baselines/SarcasmGhosh_freezed/checkpoint-1244/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1571
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SarcasmGhosh_freezed/checkpoint-2488
Configuration saved in ../../result/baselines/SarcasmGhosh_freezed/checkpoint-2488/config.json
Model weights saved in ../../result/baselines/SarcasmGhosh_freezed/checkpoint-2488/pytorch_model.bin
Deleting older checkpoint [../../result/baselines/SarcasmGhosh_freezed/checkpoint-1244] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1571
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SarcasmGhosh_freezed/checkpoint-3732
Configuration sav

## 15

In [ ]:
task = list(tasks.keys())[15]
train_baseline(task)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file

Epoch,Training Loss,Validation Loss,Rmse
1,0.009300,0.020625,0.143613
2,0.007400,0.013541,0.116365
3,0.006100,0.013954,0.118127
4,0.005100,0.015877,0.126003
5,0.004200,0.014584,0.120764


***** Running Evaluation *****
  Num examples = 1044
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SentiTreeBank/checkpoint-7378
Configuration saved in ../../result/baselines/SentiTreeBank/checkpoint-7378/config.json
Model weights saved in ../../result/baselines/SentiTreeBank/checkpoint-7378/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1044
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SentiTreeBank/checkpoint-14756
Configuration saved in ../../result/baselines/SentiTreeBank/checkpoint-14756/config.json
Model weights saved in ../../result/baselines/SentiTreeBank/checkpoint-14756/pytorch_model.bin
Deleting older checkpoint [../../result/baselines/SentiTreeBank/checkpoint-7378] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1044
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SentiTreeBank/checkpoint-22134
Configuration saved in ../../result/baselines/SentiTreeBank/checkpoin

In [ ]:
task = list(tasks.keys())[15]
train_baseline(task, freeze_bert=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file

Epoch,Training Loss,Validation Loss,Rmse
1,0.026600,0.046444,0.215510
2,0.024200,0.042352,0.205795
3,0.023700,0.041683,0.204163
4,0.023100,0.042541,0.206255
5,0.024000,0.041688,0.204176


***** Running Evaluation *****
  Num examples = 1044
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SentiTreeBank_freezed/checkpoint-7378
Configuration saved in ../../result/baselines/SentiTreeBank_freezed/checkpoint-7378/config.json
Model weights saved in ../../result/baselines/SentiTreeBank_freezed/checkpoint-7378/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1044
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SentiTreeBank_freezed/checkpoint-14756
Configuration saved in ../../result/baselines/SentiTreeBank_freezed/checkpoint-14756/config.json
Model weights saved in ../../result/baselines/SentiTreeBank_freezed/checkpoint-14756/pytorch_model.bin
Deleting older checkpoint [../../result/baselines/SentiTreeBank_freezed/checkpoint-7378] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1044
  Batch size = 32
Saving model checkpoint to ../../result/baselines/SentiTreeBank_freezed/checkpoint-22134
Confi

## 16

In [ ]:
task = list(tasks.keys())[16]
train_baseline(task)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.169300,0.135504,0.956631,0.956784,0.950100,0.963563
2,0.077000,0.091944,0.971760,0.971602,0.973577,0.969636
3,0.018900,0.152957,0.966717,0.967131,0.951961,0.982794
4,0.006200,0.198445,0.968230,0.968358,0.961117,0.975709
5,0.001900,0.218333,0.967726,0.967904,0.959245,0.976721


***** Running Evaluation *****
  Num examples = 1983
  Batch size = 32
Saving model checkpoint to ../../result/baselines/ShortHumor/checkpoint-1182
Configuration saved in ../../result/baselines/ShortHumor/checkpoint-1182/config.json
Model weights saved in ../../result/baselines/ShortHumor/checkpoint-1182/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1983
  Batch size = 32
Saving model checkpoint to ../../result/baselines/ShortHumor/checkpoint-2364
Configuration saved in ../../result/baselines/ShortHumor/checkpoint-2364/config.json
Model weights saved in ../../result/baselines/ShortHumor/checkpoint-2364/pytorch_model.bin
Deleting older checkpoint [../../result/baselines/ShortHumor/checkpoint-1182] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1983
  Batch size = 32
Saving model checkpoint to ../../result/baselines/ShortHumor/checkpoint-3546
Configuration saved in ../../result/baselines/ShortHumor/checkpoint-3546/config.json
Model weight

In [ ]:
singletaskbert.load_state_dict(torch.load('../../result/baselines/ShortHumor/checkpoint-5910/pytorch_model.bin'))

<All keys matched successfully>

In [ ]:
trainer = Trainer(model=singletaskbert, args=training_args, compute_metrics=compute_metrics)

In [ ]:
trainer.evaluate(train_dataset)

***** Running Evaluation *****
  Num examples = 37801
  Batch size = 32


{'eval_loss': 0.00048118969425559044,
 'eval_accuracy': 0.9998941826935795,
 'eval_f1': 0.9998942414467769,
 'eval_precision': 0.9998413705583756,
 'eval_recall': 0.9999471179270227,
 'eval_runtime': 209.4594,
 'eval_samples_per_second': 180.469,
 'eval_steps_per_second': 5.643}

In [ ]:
task = list(tasks.keys())[16]
train_baseline(task, freeze_bert=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/j

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/jz17d/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.618600,0.573215,0.702975,0.737171,0.659218,0.836032
2,0.575800,0.541005,0.729198,0.748242,0.696943,0.807692
3,0.550400,0.535516,0.726677,0.756951,0.679549,0.854251
4,0.548000,0.518722,0.745840,0.756757,0.723247,0.793522
5,0.542300,0.518570,0.739284,0.755786,0.708592,0.809717


***** Running Evaluation *****
  Num examples = 1983
  Batch size = 32
Saving model checkpoint to ../../result/baselines/ShortHumor_freezed/checkpoint-1182
Configuration saved in ../../result/baselines/ShortHumor_freezed/checkpoint-1182/config.json
Model weights saved in ../../result/baselines/ShortHumor_freezed/checkpoint-1182/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1983
  Batch size = 32
Saving model checkpoint to ../../result/baselines/ShortHumor_freezed/checkpoint-2364
Configuration saved in ../../result/baselines/ShortHumor_freezed/checkpoint-2364/config.json
Model weights saved in ../../result/baselines/ShortHumor_freezed/checkpoint-2364/pytorch_model.bin
Deleting older checkpoint [../../result/baselines/ShortHumor_freezed/checkpoint-1182] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1983
  Batch size = 32
Saving model checkpoint to ../../result/baselines/ShortHumor_freezed/checkpoint-3546
Configuration saved in ../../resu

### debug

In [ ]:
task = list(tasks.keys())[16]
freeze_bert = True

torch.cuda.empty_cache()
model = None
trainer = None 
num_labels = tasks[task]

data_folder = '../../data/xslue'
train_dataset = MyDataset(f'{data_folder}/processed/train/{task}.tsv')
test_dataset = MyDataset(f'{data_folder}/processed/test/{task}.tsv')
valid_dataset = MyDataset(f'{data_folder}/processed/dev/{task}.tsv')

singletaskbert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels) 
if freeze_bert:
    for param in singletaskbert.bert.parameters():
        param.requires_grad = False
result_folder = '../../result'
training_args = TrainingArguments(
    output_dir=f"{result_folder}/baselines/{task+'_freezed' if freeze_bert else task}",   # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=batch_size,  # batch size per device during training
    per_device_eval_batch_size=batch_size,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=f"{result_folder}/baselines/{task+'_freezed' if freeze_bert else task}/logs",  # directory for storing logs
#         logging_first_step = True, 
#         logging_steps=500,               # log & save weights each logging_steps
#         save_steps=500,
    evaluation_strategy="epoch",     # evaluate each `logging_steps`
    save_total_limit = 1,
    save_strategy = 'epoch',
    load_best_model_at_end=True, # decide on loss
)


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


trainer = Trainer(
    model=singletaskbert,   # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
#         test_dataset=test_dataset,            # test dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
singletaskbert.bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
list(singletaskbert.bert.pooler.parameters())

[Parameter containing:
 tensor([[-0.0013, -0.0381, -0.0158,  ...,  0.0244, -0.0008,  0.0240],
         [ 0.0020,  0.0151,  0.0033,  ...,  0.0180, -0.0023,  0.0231],
         [-0.0386,  0.0145,  0.0621,  ...,  0.0374, -0.0105, -0.0395],
         ...,
         [-0.0111,  0.0136,  0.0541,  ...,  0.0666,  0.0017, -0.0090],
         [ 0.0001,  0.0024, -0.0125,  ...,  0.0046, -0.0014, -0.0079],
         [ 0.0415,  0.0751,  0.0305,  ...,  0.0317,  0.0479,  0.0080]],
        device='cuda:0'),
 Parameter containing:
 tensor([-3.5976e-02, -3.8954e-03,  5.1814e-02,  2.2247e-02, -4.9372e-03,
         -1.1203e-03,  2.6339e-02,  9.0639e-03,  3.8723e-02, -9.6557e-02,
          2.3311e-02, -2.0086e-02,  6.5490e-02, -5.1680e-02,  6.6156e-02,
         -1.4721e-02, -4.5325e-03, -8.4059e-04,  1.3027e-02, -2.9770e-02,
          4.3693e-02, -1.4292e-02,  4.5489e-02,  4.5346e-03,  7.6859e-03,
         -4.0926e-02, -1.1412e-02,  6.6037e-02,  6.7376e-02,  4.2859e-02,
         -2.6975e-02,  3.5281e-05, -8.1483e

In [ ]:
list(singletaskbert.bert.encoder.layer[-1].parameters())

[Parameter containing:
 tensor([[-0.0414, -0.0009, -0.0389,  ..., -0.0488,  0.0483, -0.0530],
         [-0.0649, -0.0347, -0.0141,  ...,  0.0332, -0.0758,  0.0059],
         [-0.0320,  0.0619, -0.0941,  ...,  0.0221, -0.0589, -0.0778],
         ...,
         [ 0.0453,  0.0145, -0.0392,  ..., -0.0525, -0.0572,  0.0079],
         [ 0.0710, -0.0197,  0.0472,  ...,  0.0008, -0.0789, -0.0107],
         [-0.0549,  0.0375,  0.0308,  ...,  0.0273,  0.0536, -0.0444]],
        device='cuda:0'),
 Parameter containing:
 tensor([ 5.0130e-02,  1.1887e-01,  2.1868e-01,  8.7210e-02, -2.0477e-01,
          2.2648e-01, -3.4360e-01,  6.4057e-02, -4.1554e-01,  1.1557e-01,
          2.8522e-01, -1.0724e-01,  2.1037e-01,  6.5960e-02, -8.9728e-02,
         -6.1263e-02,  1.1344e-01, -1.9265e-01, -1.2109e-02, -2.4905e-01,
          7.1558e-02, -2.0746e-01, -1.8167e-01, -1.9616e-02,  5.7593e-02,
         -4.0425e-02,  3.1487e-01, -1.6641e-01,  3.6615e-01,  2.8458e-02,
         -8.9834e-02, -4.3720e-02, -1.2218e

In [ ]:
singletaskbert.load_state_dict(torch.load('../../result/baselines/ShortHumor_freezed/checkpoint-5910/pytorch_model.bin'))

<All keys matched successfully>

In [ ]:
trainer = Trainer(model=singletaskbert, args=training_args, compute_metrics=compute_metrics)

In [ ]:
trainer.evaluate(train_dataset)

***** Running Evaluation *****
  Num examples = 37801
  Batch size = 32


{'eval_loss': 0.5187092423439026,
 'eval_accuracy': 0.7524139573027169,
 'eval_f1': 0.7676456714416942,
 'eval_precision': 0.7234779353268754,
 'eval_recall': 0.8175568482284505,
 'eval_runtime': 212.9343,
 'eval_samples_per_second': 177.524,
 'eval_steps_per_second': 5.551}